In [190]:
# Import required modules
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import numpy as np

#https://bettingdata.com/nfl/odds?week=5&season=2021&seasontype=1&state=WORLD&league=nfl
df_team_index = pd.read_csv("Team Index.csv")
bets_df = pd.read_csv("BettingOdds.csv")

bets_df['FAVORITE_SCORE'] = bets_df['TOTAL']/2 - bets_df['SPREAD']
bets_df['UNDERDOG_SCORE'] = bets_df['TOTAL']/2 + bets_df['SPREAD']
bets_df['FAVORITE'] = bets_df['FAVORITE'].str.replace("at ", "")
bets_df['UNDERDOG'] = bets_df['UNDERDOG'].str.replace("at ", "")

test_df = bets_df[bets_df['WEEK']==week_num+1]
test_df1 = test_df[['FAVORITE','UNDERDOG','SPREAD','FAVORITE_SCORE']].rename(columns={"FAVORITE": "OFF_TEAM", "UNDERDOG": "DEF_TEAM", "FAVORITE_SCORE": "BETS_SCORE"})
test_df2 = test_df[['FAVORITE','UNDERDOG','SPREAD','UNDERDOG_SCORE']].rename(columns={"UNDERDOG": "OFF_TEAM", "FAVORITE": "DEF_TEAM", "UNDERDOG_SCORE": "BETS_SCORE"})
test_df2['SPREAD'] = -test_df2['SPREAD']
test_df = pd.concat([test_df1, test_df2])

favorites_df = bets_df[['FAVORITE','FAVORITE_SCORE','SPREAD','WEEK']]
favorites_df.rename(columns={"FAVORITE": "Team", "FAVORITE_SCORE": "BETS_SCORE"}, inplace=True)

underdog_df = bets_df[['UNDERDOG','UNDERDOG_SCORE','SPREAD','WEEK']]
underdog_df['SPREAD'] = -underdog_df['SPREAD']
underdog_df.rename(columns={"UNDERDOG": "Team", "UNDERDOG_SCORE": "BETS_SCORE"}, inplace=True)

bets_df = pd.concat([favorites_df,underdog_df], ignore_index=True)
bets_df = pd.merge(bets_df, df_team_index, on='Team')
bets_df = bets_df[['Abb','WEEK','SPREAD','BETS_SCORE']]
bets_df.rename(columns={"Abb": "Team"}, inplace=True)
bets_df.head()

C:\Users\onine\anaconda3\lib\site-packages\pandas\core\frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
<ipython-input-190-1f8c9cabb147>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  underdog_df['SPREAD'] = -underdog_df['SPREAD']


,Team,WEEK,SPREAD,BETS_SCORE
0,TB,1.0,-9.0,35.25
1,TB,2.0,-13.0,39.00
2,TB,3.0,-0.5,28.00
3,TB,4.0,-6.5,31.25
4,TB,5.0,-11.0,35.00


In [208]:
# Create variables for scraped data
OFF_TEAM = []
DEF_TEAM = []
FPTS = []
WEEK = []
pos = 'DEF'
pos_dict = {'QB':1,'RB':2,'WR':3,'TE':4,'K':7,'DEF':8}
pos_key = pos_dict[pos]
week_num = 7

for i in range(week_num):

    url = "https://fantasy.nfl.com/research/pointsagainst?position=" + str(pos_key) + "&statCategory=pointsAgainst&statSeason=2021&statType=weekPointsAgainst&statWeek=" + str(i+1)
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'lxml')
    time.sleep(1)

    # Create an object of the first object that is class=data compact
    table = soup.find('tbody')

    # Find all the <tr> tag pairs, skip the first one, then for each.
    for row in table.find_all('tr'):
        
        # Create a variable of all the <td> tag pairs in each <tr> tag pair,
        #col = row.find('td').text

        DEF_TEAM.append(row.find_all('td')[0].text)
        OFF_TEAM.append(row.find_all('td')[1].text)
        FPTS.append(row.find_all('td')[2].text)
        WEEK.append(i+1)

    # Create a variable of the value of the columns
col = {'OFF_TEAM': OFF_TEAM,
       'DEF_TEAM': DEF_TEAM,
       'FPTS': FPTS,
       'WEEK': WEEK,
       }

    # Create a dataframe from the columns variable
df = pd.DataFrame(col)
df['POSITION'] = df['DEF_TEAM'].apply(lambda x: x.split('vs ')[1])
df['DEF_TEAM'] = df['DEF_TEAM'].apply(lambda x: x.split('vs ')[0]).str.replace(" Defense", "")
df['OFF_TEAM'] = df['OFF_TEAM'].apply(lambda x: x.replace("@", ""))
df['OFF_TEAM'] = df['OFF_TEAM'].apply(lambda x: 'WSH' if x=='WAS' else x)
df['OFF_TEAM'] = df['OFF_TEAM'].apply(lambda x: 'LAR' if x=='LA' else x)
df['FPTS'] = df['FPTS'].astype(float).astype(int)
df = df[["WEEK", "OFF_TEAM", "DEF_TEAM" , "FPTS"]]

df.to_csv("FF_Data", index=False)
df = pd.merge(df, df_team_index, left_on='DEF_TEAM', right_on='FullName')
df.drop(['DEF_TEAM'], axis=1, inplace=True)
df.rename(columns={"Abb": "DEF_TEAM"}, inplace=True)
OFF_df = df.groupby(by='OFF_TEAM').mean().reset_index()[['OFF_TEAM','FPTS']]
DEF_df = df.groupby(by='DEF_TEAM').mean().reset_index()[['DEF_TEAM','FPTS']]
df = pd.merge(df, OFF_df, on='OFF_TEAM', suffixes=[None,"_OFF_AVG"])
df = pd.merge(df, DEF_df, on='DEF_TEAM', suffixes=[None,"_DEF_AVG"])
df2 = pd.merge(bets_df, df, right_on=['OFF_TEAM','WEEK'], left_on=['Team','WEEK'], how='inner')
df2 = df2[['WEEK','OFF_TEAM','FPTS_OFF_AVG','DEF_TEAM','FPTS_DEF_AVG','SPREAD','BETS_SCORE','FPTS']]
df2.head(3)

,WEEK,OFF_TEAM,FPTS_OFF_AVG,DEF_TEAM,FPTS_DEF_AVG,SPREAD,BETS_SCORE,FPTS
0,1.0,TB,9.0,DAL,2.285714,-9.0,35.25,2
1,2.0,TB,9.0,ATL,7.000000,-13.0,39.00,19
2,3.0,TB,9.0,LAR,3.000000,-0.5,28.00,0


In [209]:
from catboost import CatBoostRegressor

X = df2[['FPTS_OFF_AVG','FPTS_DEF_AVG','SPREAD','BETS_SCORE']]
y = df2['FPTS']

from sklearn.model_selection import train_test_split
X_train, X_validation, y_train, y_validation = train_test_split(X, y, train_size=0.7, random_state=1234)

#categorical_features_indices = np.where(X.dtypes != float)[0]

#importing library and building model
from catboost import CatBoostRegressor
model=CatBoostRegressor(iterations=1000, depth=3, learning_rate=0.01, loss_function='RMSE', use_best_model=True )
model.fit(X_train, y_train, eval_set=(X_validation, y_validation), plot=True) 
#model.fit(X_train, y_train, cat_features=categorical_features_indices, eval_set=(X_validation, y_validation),plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 4.9330399	test: 5.6366091	best: 5.6366091 (0)	total: 420us	remaining: 420ms
1:	learn: 4.9268311	test: 5.6305681	best: 5.6305681 (1)	total: 775us	remaining: 387ms
2:	learn: 4.9193218	test: 5.6236327	best: 5.6236327 (2)	total: 1.12ms	remaining: 371ms
3:	learn: 4.9090925	test: 5.6099191	best: 5.6099191 (3)	total: 1.48ms	remaining: 368ms
4:	learn: 4.8968268	test: 5.5978587	best: 5.5978587 (4)	total: 1.85ms	remaining: 369ms
5:	learn: 4.8885206	test: 5.5890262	best: 5.5890262 (5)	total: 2.11ms	remaining: 349ms
6:	learn: 4.8800230	test: 5.5796583	best: 5.5796583 (6)	total: 2.39ms	remaining: 340ms
7:	learn: 4.8739246	test: 5.5718025	best: 5.5718025 (7)	total: 2.62ms	remaining: 325ms
8:	learn: 4.8640884	test: 5.5610357	best: 5.5610357 (8)	total: 2.88ms	remaining: 317ms
9:	learn: 4.8545584	test: 5.5506057	best: 5.5506057 (9)	total: 3.17ms	remaining: 314ms
10:	learn: 4.8488971	test: 5.5447382	best: 5.5447382 (10)	total: 3.48ms	remaining: 313ms
11:	learn: 4.8395113	test: 5.5349721	best: 

647:	learn: 3.5307003	test: 4.4665005	best: 4.4613982 (633)	total: 139ms	remaining: 75.6ms
648:	learn: 3.5293324	test: 4.4663334	best: 4.4613982 (633)	total: 139ms	remaining: 75.4ms
649:	learn: 3.5281038	test: 4.4670542	best: 4.4613982 (633)	total: 140ms	remaining: 75.2ms
650:	learn: 3.5267802	test: 4.4668824	best: 4.4613982 (633)	total: 140ms	remaining: 75ms
651:	learn: 3.5259269	test: 4.4670034	best: 4.4613982 (633)	total: 140ms	remaining: 74.8ms
652:	learn: 3.5246257	test: 4.4671909	best: 4.4613982 (633)	total: 140ms	remaining: 74.6ms
653:	learn: 3.5235483	test: 4.4674964	best: 4.4613982 (633)	total: 141ms	remaining: 74.4ms
654:	learn: 3.5223975	test: 4.4673975	best: 4.4613982 (633)	total: 141ms	remaining: 74.2ms
655:	learn: 3.5214881	test: 4.4680586	best: 4.4613982 (633)	total: 141ms	remaining: 74ms
656:	learn: 3.5204963	test: 4.4676131	best: 4.4613982 (633)	total: 141ms	remaining: 73.8ms
657:	learn: 3.5196329	test: 4.4688017	best: 4.4613982 (633)	total: 142ms	remaining: 73.6ms
658

In [210]:
test_df2 = pd.merge(test_df, df_team_index, left_on='OFF_TEAM', right_on='Team')
test_df2.drop(['OFF_TEAM'], axis=1, inplace=True)
test_df2.rename(columns={"Abb": "OFF_TEAM"}, inplace=True)
test_df2 = pd.merge(test_df2, df_team_index, left_on='DEF_TEAM', right_on='Team')
test_df2.drop(['DEF_TEAM'], axis=1, inplace=True)
test_df2.rename(columns={"Abb": "DEF_TEAM"}, inplace=True)

test_df2 = pd.merge(test_df2, OFF_df, on='OFF_TEAM', suffixes=[None,"_OFF_AVG"])
test_df2 = pd.merge(test_df2, DEF_df, on='DEF_TEAM', suffixes=[None,"_DEF_AVG"])
test_df2.rename(columns={"FPTS": "FPTS_OFF_AVG"}, inplace=True)

test_df2 = test_df2[['OFF_TEAM','FPTS_OFF_AVG','DEF_TEAM','FPTS_DEF_AVG','SPREAD','BETS_SCORE']]
test_df3 = test_df2[['FPTS_OFF_AVG','FPTS_DEF_AVG','SPREAD','BETS_SCORE']]

test_df2['pred'] = model.predict(test_df3)
test_df2.sort_values(by='pred', ascending=False).head(50)

,OFF_TEAM,FPTS_OFF_AVG,DEF_TEAM,FPTS_DEF_AVG,SPREAD,BETS_SCORE,pred
2,BUF,12.000000,MIA,7.714286,-14.0,38.25,12.378688
7,CIN,5.857143,NYJ,9.571429,-10.0,31.50,10.658430
8,LAR,6.857143,HOU,9.142857,-14.5,38.25,10.359630
16,CAR,6.000000,ATL,7.000000,3.0,20.50,7.950245
6,IND,8.428571,TEN,5.714286,-2.5,27.75,7.614619
18,CHI,6.285714,SF,4.857143,4.0,15.75,6.701999
14,KC,3.428571,NYG,6.714286,-9.5,35.50,6.677712
19,PIT,5.666667,CLE,4.285714,3.5,17.50,6.583338
11,DEN,6.428571,WSH,5.571429,-3.0,25.25,6.422890
28,DAL,9.500000,MIN,4.000000,2.5,24.50,6.403628
